In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
import numpy as np
from scipy.special import softmax
import pandas as pd

In [2]:
data = pd.read_csv("/kaggle/input/sentiment-analysis/sample_headspace_user_tweets.csv")

In [3]:
data.head()

,Unnamed: 0,user,tweet,source
0,0,Gra6011,"RT @HBOMaxLA: @Gra6011, por tu gran fortaleza,...",Twitter for Android
1,1,Gra6011,RT @asoiafsp: ¿Team Daemon o Team Criston? \n\...,Twitter for Android
2,2,Gra6011,@despoleo Canadiense de Terrabusi,Twitter for Android
3,3,Gra6011,RT @MoskitaMuertaOk: ¿Quién merece ganar #ElHo...,Twitter for Android
4,4,Gra6011,RT @ElHotelTrivia: ¿TRAICIÓN? ¿JUEGO? 😱\n\n¿Ba...,Twitter for Android


In [4]:
tweets = data['tweet']

In [5]:
len(tweets)

202

In [6]:
def sentiment_analyze(tweet):
    # Preprocess Tweet
    words = tweet.split(' ')
    words = ['@user' if (w.startswith('@') and len(w) > 1) else w for w in words]
    tweet_preprocessed = " ".join(words)
    print(f'Tweet (preprocessed): {tweet_preprocessed}')
    
    # Initialize Model
    roberta_url = 'cardiffnlp/twitter-roberta-base-sentiment'
    model = AutoModelForSequenceClassification.from_pretrained(roberta_url)
    tokenizer = AutoTokenizer.from_pretrained(roberta_url)
    labels = ['Negative', 'Neutral', 'Positive']
    
    # Run Model(Tweet)
    tweet_encoded = tokenizer(tweet_preprocessed, return_tensors='pt')
    #output = model(tweet_encoded['input_ids'], tweet_encoded['attention_mask'])
    output = model(**tweet_encoded)
    
    # Output
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for i in range(len(scores)):
        print(f'{labels[i]}: {scores[i]}')
        
    return labels, scores

In [19]:
for tweet in tweets:
    sentiment_analyze(tweet)

Tweet (preprocessed): RT @user @user por tu gran fortaleza, y por la conexión que has forjado con el reino, eres Príncipe/Princesa de Dragonstone de la…
Negative: 0.02373499982059002
Neutral: 0.8998293876647949
Positive: 0.07643555104732513
Tweet (preprocessed): RT @user ¿Team Daemon o Team Criston? 

RT- Criston
Me Gusta- Daemon 
#HouseOfTheDragon https://t.co/EUlo0vFVKU
Negative: 0.053604964166879654
Neutral: 0.8958569765090942
Positive: 0.05053802207112312
Tweet (preprocessed): @user Canadiense de Terrabusi
Negative: 0.11644371598958969
Neutral: 0.7854296565055847
Positive: 0.09812667220830917
Tweet (preprocessed): RT @user ¿Quién merece ganar #ElHotelDeLosFamosos?
RT Alex Caniggia 
FAV Martín Salwe https://t.co/kKhDWvjfak
Negative: 0.06900780647993088
Neutral: 0.8851062059402466
Positive: 0.04588595777750015
Tweet (preprocessed): RT @user ¿TRAICIÓN? ¿JUEGO? 😱

¿Bancas a Lissa Vera? 

🔁 SI
❤️ NO

#ElHotelDeLosFamosos https://t.co/G2l4vdwXZD
Negative: 0.11163979768753052
Neutral: 0.7

['Negative', 'Neutral', 'Positive']